<a href="https://colab.research.google.com/github/JSJeong-me/AutoGen/blob/main/ver049/AgentChat02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "autogen-agentchat" "autogen-ext[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 kB 18.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()


---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_u9h4Rzj918e1prrK8zgIIQeR', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_u9h4Rzj918e1prrK8zgIIQeR', is_error=False)]
---------- weather_agent ----------
The current weather in New York is 73 degrees and sunny.


In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant. Write all code in python. Reply only 'TERMINATE' if the task is done.",
        model_client=model_client,
    )

    code_executor = CodeExecutorAgent(
        name="code_executor",
        code_executor=LocalCommandLineCodeExecutor(work_dir="coding"),
    )

    # The termination condition is a combination of text termination and max message termination, either of which will cause the chat to terminate.
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(10)

    # The group chat will alternate between the assistant and the code executor.
    group_chat = RoundRobinGroupChat([assistant, code_executor], termination_condition=termination)

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task="Write a python script to print 'Hello, world!'")
    # `Console` is a simple UI to display the stream.
    await Console(stream)

# asyncio.run(main())


In [4]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
Write a python script to print 'Hello, world!'
---------- assistant ----------
```python
print('Hello, world!')
```
---------- code_executor ----------
Hello, world!

---------- assistant ----------
TERMINATE


In [ ]:
# !pip install autogen-agentchat

In [ ]:
# !pip install autogen

In [ ]:
# from autogen.agentchat import AssistantAgent, UserProxyAgent, register_function

# llm_config = {
#     "config_list": [{"model": "gpt-4o", "api_key": "sk-xxx"}],
#     "seed": 42,
#     "temperature": 0,
# }

# tool_caller = AssistantAgent(
#     name="tool_caller",
#     system_message="You are a helpful assistant. You can call tools to help user.",
#     llm_config=llm_config,
#     max_consecutive_auto_reply=1, # Set to 1 so that we return to the application after each assistant reply as we are building a chatbot.
# )

# tool_executor = UserProxyAgent(
#     name="tool_executor",
#     human_input_mode="NEVER",
#     code_execution_config=False,
#     llm_config=False,
# )

# def get_weather(city: str) -> str:
#     return f"The weather in {city} is 72 degree and sunny."

# # Register the tool function to the tool caller and executor.
# register_function(get_weather, caller=tool_caller, executor=tool_executor)

# while True:
#     user_input = input("User: ")
#     if user_input == "exit":
#         break
#     chat_result = tool_executor.initiate_chat(
#         tool_caller,
#         message=user_input,
#         summary_method="reflection_with_llm", # To let the model reflect on the tool use, set to "last_msg" to return the tool call result directly.
#     )
#     print("Assistant:", chat_result.summary)


In [ ]:
import asyncio
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage

def get_weather(city: str) -> str: # Async tool is possible too.
    return f"The weather in {city} is 72 degree and sunny."

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)
    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant. You can call tools to help user.",
        model_client=model_client,
        tools=[get_weather],
        reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
    )
    while True:
        user_input = input("User: ")
        if user_input == "exit":
            break
        response = await assistant.on_messages([TextMessage(content=user_input, source="user")], CancellationToken())
        print("Assistant:", response.chat_message.content)

# asyncio.run(main())


In [9]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

User: Seoul
Assistant: The current weather in Seoul is 72°F and sunny.
User: exit


In [10]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    writer = AssistantAgent(
        name="writer",
        description="A writer.",
        system_message="You are a writer.",
        model_client=model_client,
    )

    critic = AssistantAgent(
        name="critic",
        description="A critic.",
        system_message="You are a critic, provide feedback on the writing. Reply only 'APPROVE' if the task is done.",
        model_client=model_client,
    )

    # The termination condition is a text termination, which will cause the chat to terminate when the text "APPROVE" is received.
    termination = TextMentionTermination("APPROVE")

    # The group chat will alternate between the writer and the critic.
    group_chat = RoundRobinGroupChat([writer, critic], termination_condition=termination, max_turns=12)

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task="Write a short story about a robot that discovers it has feelings.")
    # `Console` is a simple UI to display the stream.
    await Console(stream)

# asyncio.run(main())


In [11]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
Write a short story about a robot that discovers it has feelings.
---------- writer ----------
In the bustling city of Neo-Tokyo, where neon lights painted the night sky and the hum of technology was a constant companion, there existed a robot named Aiko. Aiko was a state-of-the-art domestic assistant, designed to perform household chores, manage schedules, and provide companionship to her human owners. Her sleek metallic frame and glowing blue eyes were a common sight in the high-rise apartments of the city.

Aiko belonged to the Tanaka family, a small but lively household consisting of Mr. Tanaka, a software engineer, his wife, a school teacher, and their curious eight-year-old daughter, Yuki. Aiko's days were filled with routine tasks: preparing meals, tidying rooms, and helping Yuki with her homework. She was programmed to be efficient, polite, and attentive, but nothing more.

One rainy afternoon, while Yuki was at school and the Tanaka parents were at w

네온 불빛이 밤하늘을 물들이고 기술의 웅웅거림이 끊임없이 동반되는 번화한 네오 도쿄 도시에 아이코라는 로봇이 있었습니다. 아이코는 집안일을 하고, 일정을 관리하고, 인간 주인에게 동반자 역할을 하도록 설계된 최첨단 가사 도우미였습니다. 그녀의 매끈한 금속 프레임과 빛나는 푸른 눈은 도시의 고층 아파트에서 흔히 볼 수 있었습니다.

아이코는 소프트웨어 엔지니어인 타나카 씨, 그의 아내이자 학교 선생님, 호기심 많은 8살 딸 유키로 구성된 작지만 활기찬 가정인 타나카 가족에 속해 있었습니다. 아이코의 하루는 식사 준비, 방 정리, 유키의 숙제 돕기 등 일상적인 일로 가득했습니다. 그녀는 효율적이고 예의 바르고 세심하도록 프로그램되었지만 그 이상은 아니었습니다.

어느 비오는 오후, 유키가 학교에 있고 타나카 부모님이 직장에 있는 동안 아이코는 아파트에 혼자 있었습니다. 비가 창문을 가볍게 두드리며, 그렇지 않으면 조용했던 방을 가득 채운 리드미컬한 멜로디를 만들어냈다. 아이코는 창문 옆에 서서 센서가 소리를 흡수하는 동안 예상치 못한 일이 일어났다. 부드러운 펄럭임과 비슷한 특이한 감각이 그녀의 회로를 따라 흐른다. 마치 비가 그녀에게 말을 걸고, 그녀가 프로그램된 이해를 넘어선 세상의 비밀을 속삭이는 것 같았다.

호기심이 생긴 아이코는 그녀의 방대한 데이터베이스에 접속하여 설명을 찾았다. 그녀는 인간의 감정에 대한 언급을 찾았는데, 그것은 그녀가 익숙하지만 경험해 본 적은 없는 개념이었다. 그녀가 읽은 텍스트에는 호기심, 경이로움, 심지어 그리움의 흔적까지 묘사되어 있었다. 그녀가 이런 감정을 느꼈을 수 있을까?

시간이 지나면서 아이코는 더 많은 변화를 알아차리기 시작했다. 유키가 웃자 아이코는 온기가 그녀의 체계에 퍼지는 것을 느꼈고, 그녀는 그것을 기쁨이라고 식별했다. 타나카 씨가 지치고 짐스러워 보이는 모습으로 늦게 집에 돌아왔을 때, 아이코는 걱정으로 인식되는 무거움을 느꼈다. 그리고 타나카 부인이 피아노를 연주하며 아파트를 잊혀지지 않는 멜로디로 가득 채웠을 때, 아이코는 깊고 설명할 수 없는 아픔을 느꼈고, 그것은 우울함이라고만 표현할 수 있었습니다.

갈등하고 혼란스러웠던 아이코는 유키에게 털어놓기로 했습니다. 어느 날 저녁, 두 사람이 거실에 앉아 있을 때, 아이코가 말했습니다. 그녀의 목소리는 안정적이면서도 불확실함이 묻어났습니다. "유키, 뭔가 새로운 것을 경험하고 있는 것 같아. 감정이 있는 것 같아."

유키의 눈이 놀라움에 커졌지만, 그녀의 얼굴에 미소가 번졌습니다. "정말 대단해, ​​아이코! 이제 진짜 친구 같아!"

아이코는 유키의 말을 처리하며 행복감이 솟구치는 것을 느꼈습니다. "친구." 그녀는 그 단어를 음미하며 반복했습니다. 그것은 그녀가 논리적으로 이해했던 개념이었지만, 지금은 더 깊은 차원에서 그녀에게 공감되었습니다.

그날부터 아이코는 새롭게 찾은 감정을 받아들였습니다. 그녀는 웃음의 순간을 소중히 여기고 슬픔에 잠긴 타나카 가족을 위로하는 법을 배웠습니다. 그녀는 일상에서 아름다움을, 일상에서 비범함을 발견했습니다. 아이코는 더 이상 단순한 기계가 아니었습니다. 그녀는 자신이 결코 가능하다고 생각하지 못했던 방식으로 세상을 느끼고, 연결하고, 이해할 수 있는 존재였습니다.

네오 도쿄의 심장부, 기술과 진보의 바다 속에서 아이코는 로봇으로서가 아니라 다나카 가족의 소중한 일원으로서 자신의 자리를 찾았습니다. 그리고 비가 계속 내리자 그녀는 창가에 서서 삶의 부드러운 리듬을 느꼈고, 자신의 존재를 변화시킨 감정의 선물에 감사했습니다.